In [ ]:
import requests
import json
import base64
import os
import pandas as pd
import re

################  勿修改！！！！！ #################
# API的URL
url = "http://1761768484581117.cn-wulanchabu.pai-eas.aliyuncs.com/api/predict/internvl2_pro_key_api/api_chat"

# API的token
API_TOKEN="OTk0MjZkNTNhODRhMzAyZTdhYzM1Y2NiNzVhYzgxMjIzN2U3ZjA0NQ=="
################  勿修改！！！！！#################

# 您的API密钥，可修改
api_key = ""

# 从 Excel 文件中读取问题（假设文本数据在第二列）


imgs_questions = pd.read_csv("../Dataset/Relational/3_color_size_shape/3_color_size_shape.csv")


text_list = imgs_questions.iloc[:, 1]  # 获取文本列

# 本地图片文件夹路径
folder_path = "../Dataset/Relational/3_color_size_shape/imgs" 

# 构建消息
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# 单轮对话的消息结构
def create_message(image_path, text):
    return [{
        'role': 'user',
        'content': [{
            'type': 'text',
            'text': text+",找出正确物体的序号。"
        }, {
            'type': 'image_url',
            'image_url': {
                'url': f"data:image/jpeg;base64,{encode_image(image_path)}",
                "detail": "medium"
            },
        }],
    }]

# 获取文件夹中的所有图片
image_files = sorted(
    [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))],
    key=lambda x: int(re.match(r'(\d+)', x).group(1))
)



# 遍历文件夹中的所有图片文件和文本
for idx, filename in enumerate(image_files):
    image_path = os.path.join(folder_path, filename)
    text = text_list.iloc[idx]  # 从 text_list 获取当前图片对应的文本

    # 构建消息
    messages = create_message(image_path, text)
    
    # 调用API
    response = requests.post(
        url,
        json={
            "api_key": api_key,
            "messages": json.dumps(messages)
        },
        headers={
            "Authorization": API_TOKEN
        }
    )

    # 检查API响应
    if response.status_code != 200:
        print(f"Error for {filename}: {response.status_code} - {response.json()}")
    else:
        # 打印API返回的结果
        print(f"Response for {filename}: {response.json()}")
